In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

In [4]:
import graphlab
from graphlab import SGraph, Vertex, Edge
import re
import slate
import cPickle as pickle

In [207]:
# g = SGraph()

# verts = [Vertex(0, attr={'breed': 'labrador'}),
#              Vertex(1, attr={'breed': 'labrador'}),
#              Vertex(2, attr={'breed': 'vizsla'})]

In [7]:
g = g.add_vertices(verts)
g = g.add_edges(Edge(1, 2))

In [11]:
g.show()

In [14]:
with open('./fenaroli_flavors.txt') as f:
    flav_data = f.read()

In [15]:
text_shard = flav_data[:50000]

([A-Z]{3,50}\s?[A-Z]{3,50}\s?[A-Z]{3,50})

In [5]:
sq = re.compile('(\d?-?[A-Z]{2,50}[,-|\s]?[A-Z]{2,50}[,-|\s]?[A-Z]{2,50})')
raw_flavor_compounds = re.findall(sq, flav_data)
terms_to_remove = ['FEMA PADI','EINECS', 'EMA PADI', 'EINECS NO','FEMA PAD','JECFAN']

In [8]:
flavor_compounds = [compound.replace('\n','') for compound in raw_flavor_compounds]

In [270]:
# flavor_compounds = [compound.replace('\n','') for compound in raw_flavor_compounds if compound not in terms_to_remove]

In [9]:
cnt = Counter(flavor_compounds)
to_remove = [k for k, v in cnt.iteritems() if v > 1]

In [10]:
# to_remove.append(['EINECS NO', ])

In [11]:
flavor_comp = [comp for comp in flavor_compounds if comp not in to_remove and comp[:2] != '-C']

In [16]:
text_shard

In [41]:
with open('small_fenaroli_flavors.pdf') as f:
    doc = slate.PDF(f)

In [81]:
# doc[2]

In [42]:
joined_small_doc = ''.join(doc)

In [21]:
with open('./small_fenaroli_flavors.txt') as f:
    small_text = f.read()

In [33]:
import simplediff

In [35]:
len(joined_small_doc)

In [37]:
with open('small_text.txt', 'w') as f:
    f.write(small_text)

In [96]:
with open('joined_from_slate.txt', 'w') as f:
    f.write(joined)

In [46]:
joined_small_doc[:500]

In [7]:
with open('doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)
with open('doc_split.pkl', 'r') as f:
    split = pickle.load(f)

In [10]:
print len (split)
print len(joined)

In [12]:
for i in range(10):
    print len(split[i])

In [20]:
len_arr = np.array([len(split[i]) for i in range(len(split))])

In [30]:
_ = pd.Series(len_arr).hist(bins = 20)

In [80]:
# split[25]

In [65]:
cleaned_joined = joined.replace('\n\n', 'zxzxzxzx').replace('\n',' ').replace('.', ' ')

In [51]:
cleaned_joined[:100000]

entries I'm interested in start at split doc 17

In [106]:
cleaned_split[17:30]

In [ ]:
sq = re.compile('Natural occurrence:\s(([A-Z]([a-z]+[\s\,\;(])+[\s\,\;(]\w*)+)')


In [353]:
# sq = re.compile('Natural occurrence:\s\w*[\s\,\;(]\w*[\s\,\;(]\w*[\s\,\;(]\w*[\s\,\;(]')
sq = re.compile('Natural occurrence:\s((\w*[\s\,\;(]\w*)+)\s')
raw_flavor_compounds = re.findall(sq, cleaned_joined[400000:500000])

raw_flavor_compounds[0:50]

In [94]:
sq = re.compile('Natural occurrence: (.+?)\zxzxzxzx')
# raw_flavor_compounds = sq.findall(cleaned_joined[400000:500000])
raw_flavor_compounds = re.findall(sq, cleaned_joined[400000:500000])


In [96]:
raw_flavor_compounds[:50]